In [1]:
import numpy as np
import argparse
import cv2

from tqdm import tqdm
import mahotas
import sys
sys.path.append("/Users/rcarlson/Documents/GitHub/lasagna3/snakes")
from ops.imports_ipython import *

import skimage
import umap.umap_ as umap
import stringdb
import pandas as pd

from dask import compute, delayed
from dask.distributed import Client

/Users/rcarlson/miniconda3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Combine pretrained features

In [2]:
loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch0.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))

    
nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

[########################################] | 100% Completed | 10min 21.2s


In [6]:
import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch0.hdf', key = 'x')

Dropping 
49.51171875
% of features


In [ ]:
loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch1.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/df_26.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch1.hdf', key = 'x')

[###                                     ] | 9% Completed |  1min  9.8s

In [ ]:
loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch2.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch2.hdf', key = 'x')




loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch3.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch3.hdf', key = 'x')

In [ ]:
loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch4.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch4.hdf', key = 'x')




loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch5.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch5.hdf', key = 'x')



loc = '/home/rcarlson/mountb/m40-m42/process/xception/*.ch6.pkl'


files = natsorted(glob(loc))
    
    
# load pheno dfs
@delayed
def read_pheno(f):
        plate = f.split('feats_')[1].split('_')[0]
        well = f.split('feats_')[1].split('_')[1]
        cells = f.split('feats_')[1].split('_')[2] + '_' + f.split('feats_')[1].split('_')[3].split('.')[0]
        cellinfo = pd.read_csv('/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s.csv'%(plate, well, cells))

        plate = ['M' + s.split('/M')[1].split('/')[0] for s in cellinfo['0']]
        well = [s.split('/M')[1].split('_')[0][8:] for s in cellinfo['0']]
        site = [s.split('/M')[1].split('_')[1].split('.')[0][4:] for s in cellinfo['0']]
        cell = [s.split('/M')[1].split('.cell')[0].split('tif_')[1] for s in cellinfo['0']]

        tmp = pd.concat([pd.DataFrame([plate, well, site, cell]).transpose(), pd.read_pickle(f).astype('float16')], axis = 1)

        return tmp



from dask.diagnostics import ProgressBar
with ProgressBar():
    df = pd.concat(compute(*map(read_pheno, files), scheduler='threads'))


nchannels = 1
df.columns = ['plate', 'well', 'site', 'cell'] + ['cnn_feat' + str(c) for c in range(0,2048*nchannels)]
df.cell = df.cell.astype('int64')
df.site = df.site.astype('int64')

gene_info = pd.read_hdf('/home/rcarlson/mountb/m40-m42/DL_cellinfo.hdf')

df = pd.merge(gene_info[['plate','well','cell','site','cell_barcode_0','gene']], 
         df, how = 'right', on = ['plate','well', 'cell', 'site'])

import gc
del gene_info
gc.collect()

import math

sample = df.sample(50000, random_state = 7)
colstd = sample.iloc[:,6:].std(axis = 0)
colmean = sample.iloc[:,6:].mean(axis = 0)
colcov = colstd/colmean

infmean = [f == math.inf for f in colmean]
to_drop = sample.columns[6:][infmean | (colmean == 0) | (colstd == 0) | (colcov < 0.1)] 
# drop features with 0 SD/mean, infmean or < 10% COV.
print('Dropping ')
print(len(to_drop)/len(sample.iloc[:,6:].columns)*100)
print('% of features')

df.drop(to_drop, inplace = True, axis = 1)

df.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch6.hdf', key = 'x')

In [ ]:
df = pd.read_hdf('home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch4.hdf')

## keep genes with 30+ cells/sg, 2+ sg/gene
sgcellcts = df.groupby('cell_barcode_0').cell.count()
sgs = list(sgcellcts[sgcellcts >= 30].index)

tmp = df[['gene','cell_barcode_0']]
tmp=tmp[tmp.cell_barcode_0.isin(sgs)].groupby(['gene', 'cell_barcode_0']).head(1).groupby(['gene']).size() 
genes = list(tmp[tmp >= 2].index)

grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch4_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch4_grped.hdf', key = 'x')


df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch0.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch0_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch0_grped.hdf', key = 'x')

df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch1.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch1_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch1_grped.hdf', key = 'x')


df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch2.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch2_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch2_grped.hdf', key = 'x')

df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch3.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch3_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch3_grped.hdf', key = 'x')

df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch5.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch5_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch5_grped.hdf', key = 'x')

df = pd.read_hdf('/home/rcarlson/mountb/m40-m42/process/misc/df_feats_ch6.hdf')
grped = df[(df.gene.isin(genes)) & (df.cell_barcode_0.isin(sgs))].groupby(['gene', 'cell_barcode_0']).median().reset_index()
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch6_grped_sgonly.hdf', key = 'x')
grped = grped.groupby('gene').mean().iloc[:,1:]
grped.to_hdf('/home/rcarlson/mountb/m40-m42/process/misc/tf_df_feats_ch6_grped.hdf', key = 'x')




